In [1]:
import csv
from itertools import combinations
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import HTML, Audio, display


pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'center')

In [2]:
texts = [
    'We got few vegetables and fruits , and became fish eaters .',
    'Fifty yards ahead of her were the first of the rocks .',
    'It seemed the ordained order of things that dogs should work .',
    'The journey was continued at dawn .',
    'Was it the rendezvous of those who were striving to work his ruin .',
    'A dead man is of no use on a plantation .',
    'The Claudine was leaving next morning for Honolulu .',
    'Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.',
    'Different telescope designs perform differently, and have different strengths and weaknesses.',
    'Humans also judge distance by using the relative sizes of objects.',
    'If this is true then those who tend to think creatively, really are somehow different.',
    'But really in the grand scheme of things, this information is insignificant.',
    'He had a private jet with three king-size beds, expensive rugs, porcelain vases and a dining area.',
    'When I reached Atlanta my steadily increasing disappointment was not lessened. I found it a big, dull, red town.',
    'She woke Meg with a "Merry Christmas", and bade her see what was under her pillow. A green–covered book appeared, with the same picture inside, and a few words written by their mother, which made their one present very precious in their eyes.',
    "Does Jane know about your new job? No, and don't you dare tell her! She will be furious!",
    'Does she like ice cream or sweets? She likes any kind of ice cream. Chocolate, vanilla, strawberry, banana, the one with caramel, coconut, any you can think of!',
    'Earthquakes damage all structures, including bridges.',
    'Luckily, this kind of collapse is relatively infrequent.',
    'Was it using language that caused their brains to develop?',
    'If everyone followed a similar plan, the results would be impressive.',
    'Next, the hero overcomes obstacles on the way to facing their greatest challenge.',
    'For more than two hundred years the pessimists have been winning the public debate.',
    "It's wearing me out trying to juggle work with looking after my children and my family.",
    'Young people want to feel supported and appreciated by their company and their superiors.',
    'We start to see the evidence of early human civilisation, through cave paintings for example.',
    'In this culture a so-called "smile of respect" is seen as insincere and often regarded with suspicion.',
    'We can express complex thoughts, convey subtle emotions and communicate about some abstract concepts.',
    'The activists send a clear message to companies that people are no longer willing to accept the environmental and human cost of overconsumption.',
    'All this is thanks to his childhood in the mountains and to genetics, but it is his mental strength that sets him apart.'
]

In [3]:
huawei_phones = [
    ' W IY1 G AA1 T F Y UW1 V EH1 JH T AH0 B AH0 L Z AH0 N D F R UW1 T S  AH0 N D B IH0 K EY1 M F IH1 SH IY1 T ER0 Z  ',
    ' F IH1 F T IY0 Y AA1 R D Z AH0 HH EH1 D AH1 V HH ER1 W ER1 DH AH0 F ER1 S T AH1 V DH AH0 R AA1 K S  ',
    ' IH1 T S IY1 M D DH IY0 AO0 R D EY1 N D AO1 R D ER0 AH1 V TH IH1 NG Z DH AE1 T D AA1 G Z SH UH1 D W ER1 K  ',
    ' DH AH0 JH ER1 N IY0 W AA1 Z K AH0 N T IH1 N Y UW0 D AE1 T D AO1 N  ',
    ' W AA1 Z IH1 T DH AH0 R AA1 N D IH0 V UW2 AH1 V DH OW1 Z HH UW1 W ER1 S T R AY1 V IH0 NG T UW1 W ER1 K HH IH1 Z R UW1 AH0 N  ',
    ' AH0 D EH1 D M AE1 N IH1 Z AH1 V N OW1 Y UW1 S AA1 N AH0 P L AE2 N T EY1 SH AH0 N  ',
    ' DH AH0 K L AO0 D IY1 N W AA1 Z L IY1 V IH0 NG N EH1 K S T M AO1 R N IH0 NG F AO1 R HH AA2 N AH0 L UW1 L UW0  ',
    ' P R AA1 S IH0 K Y UW2 T ER0 Z HH AE1 V OW1 P AH0 N D AH0 M AE1 S IH0 V IH2 N V EH2 S T AH0 G EY1 SH AH0 N IH1 N T UW0 AE2 L AH0 G EY1 SH AH0 N Z AH1 V F IH1 K S IH0 NG G EY1 M Z AH0 N D IH2 L IY1 G AH0 L B EH1 T IH0 NG  ',
    ' D IH1 F ER0 AH0 N T T EH1 L AH0 S K OW2 P D IH0 Z AY1 N Z P ER0 F AO1 R M D IH1 F R AH0 N T L IY0  AH0 N D HH AE1 V D IH1 F ER0 AH0 N T S T R EH1 NG K TH S AH0 N D W IY1 K N AH0 S AH0 Z  ',
    ' HH Y UW1 M AH0 N Z AO1 L S OW0 JH AH1 JH D IH1 S T AH0 N S B AY1 Y UW1 Z IH0 NG DH AH0 R EH1 L AH0 T IH0 V S AY1 Z AH0 Z AH1 V AA1 B JH EH0 K T S  ',
    ' IH1 F DH IH1 S IH1 Z T R UW1 DH EH1 N DH OW1 Z HH UW1 T EH1 N D T UW1 TH IH1 NG K K R IY0 EY1 T IH0 V L IY0  R IH1 L IY0 AA1 R S AH1 M HH AW2 D IH1 F ER0 AH0 N T  ',
    ' B AH1 T R IH1 L IY0 IH0 N DH AH0 G R AE1 N D S K IY1 M AH1 V TH IH1 NG Z  DH IH1 S IH2 N F ER0 M EY1 SH AH0 N IH1 Z IH2 N S IH0 G N Y IH1 F IH0 K AH0 N T  ',
    ' HH IY1 HH AE1 D AH0 P R AY1 V AH0 T JH EH1 T W IH1 DH TH R IY1 K IH1 NG S AY1 Z B EH1 D Z  IH0 K S P EH1 N S IH0 V R AH1 G Z  P AO1 R S AH0 L AH0 N V EY1 S AH0 Z AH0 N D AH0 D AY1 N IH0 NG EH1 R IY0 AH0  ',
    ' W EH1 N AY1 R IY1 CH T AE0 T L AE1 N T AH0 M AY1 S T EH1 D AH0 L IY0 IH2 N K R IY1 S IH0 NG D IH2 S AH0 P OY1 N T M AH0 N T W AA1 Z N AA1 T L EH1 S AH0 N D  AY1 F AW1 N D IH1 T AH0 B IH1 G  D AH1 L  R EH1 D T AW1 N  ',
    ' SH IY1 W OW1 K M EH1 G W IH1 DH AH0 M EH1 R IY0 K R IH1 S M AH0 S  AH0 N D B EY1 D HH ER1 S IY1 W AH1 T W AA1 Z AH1 N D ER0 HH ER1 P IH1 L OW0  AH0 G R IY1 N K AH1 V ER0 D B UH1 K AH0 P IH1 R D  W IH1 DH DH AH0 S EY1 M P IH1 K CH ER0 IH2 N S AY1 D  AH0 N D AH0 F Y UW1 W ER1 D Z R IH1 T AH0 N B AY1 DH EH1 R M AH1 DH ER0  W IH1 CH M EY1 D DH EH1 R W AH1 N P R EH1 Z AH0 N T V EH1 R IY0 P R EH1 SH AH0 S IH0 N DH EH1 R AY1 Z  ',
    ' D AH1 Z JH EY1 N N OW1 AH0 B AW1 T Y AO1 R N UW1 JH AA1 B  N OW1  AH0 N D D OW1 N T Y UW1 D EH1 R T EH1 L HH ER1  SH IY1 W IH1 L B IY1 F Y UH1 R IY0 AH0 S  ',
    ' D AH1 Z SH IY1 L AY1 K AY1 S K R IY1 M AO1 R S W IY1 T S  SH IY1 L AY1 K S EH1 N IY0 K AY1 N D AH1 V AY1 S K R IY1 M  CH AO1 K L AH0 T  V AH0 N IH1 L AH0  S T R AO1 B EH2 R IY0  B AH0 N AE1 N AH0  DH AH0 W AH1 N W IH1 DH K EH1 R AH0 M AH0 L  K OW1 K AH0 N AH2 T  EH1 N IY0 Y UW1 K AE1 N TH IH1 NG K AH1 V  ',
    ' ER1 TH K W EY2 K S D AE1 M AH0 JH AO1 L S T R AH1 K CH ER0 Z  IH2 N K L UW1 D IH0 NG B R IH1 JH AH0 Z  ',
    ' L AH1 K AH0 L IY0  DH IH1 S K AY1 N D AH1 V K AH0 L AE1 P S IH1 Z R EH1 L AH0 T IH0 V L IY0 IH2 N F R IY1 K W AH0 N T  ',
    ' W AA1 Z IH1 T Y UW1 Z IH0 NG L AE1 NG G W AH0 JH DH AE1 T K AA1 Z D DH EH1 R B R EY1 N Z T UW1 D IH0 V EH1 L AH0 P  ',
    ' IH1 F EH1 V R IY0 W AH2 N F AA1 L OW0 D AH0 S IH1 M AH0 L ER0 P L AE1 N  DH AH0 R IH0 Z AH1 L T S W UH1 D B IY1 IH2 M P R EH1 S IH0 V  ',
    ' N EH1 K S T  DH AH0 HH IH1 R OW0 OW1 V ER0 K AH2 M Z AA1 B S T AH0 K AH0 L Z AA1 N DH AH0 W EY1 T UW1 F EY1 S IH0 NG DH EH1 R G R EY1 T AH0 S T CH AE1 L AH0 N JH  ',
    ' F AO1 R M AO1 R DH AE1 N T UW1 HH AH1 N D R AH0 D Y IH1 R Z DH AH0 P EH1 S AH0 M IH0 S T S HH AE1 V B IH1 N W IH1 N IH0 NG DH AH0 P AH1 B L IH0 K D AH0 B EY1 T  ',
    ' IH1 T S W EH1 R IH0 NG M IY1 AW1 T T R AY1 IH0 NG T UW1 JH AH1 G AH0 L W ER1 K W IH1 DH L UH1 K IH0 NG AE1 F T ER0 M AY1 CH IH1 L D R AH0 N AH0 N D M AY1 F AE1 M AH0 L IY0  ',
    ' Y AH1 NG P IY1 P AH0 L W AA1 N T T UW1 F IY1 L S AH0 P AO1 R T IH0 D AH0 N D AH0 P R IY1 SH IY0 EY2 T IH0 D B AY1 DH EH1 R K AH1 M P AH0 N IY2 AH0 N D DH EH1 R S UW0 P IH1 R IY0 ER0 Z  ',
    ' W IY1 S T AA1 R T T UW1 S IY1 DH IY0 EH1 V AH0 D AH0 N S AH1 V ER1 L IY0 HH Y UW1 M AH0 N S IH1 V AH0 L IH0 S EY1 SH AH0 N  TH R UW1 K EY1 V P EY1 N T IH0 NG Z F AO1 R IH0 G Z AE1 M P AH0 L  ',
    ' IH0 N DH IH1 S K AH1 L CH ER0 AH0 S OW1 K AO1 L D S M AY1 L AH1 V R IH0 S P EH1 K T IH1 Z S IY1 N AE1 Z IH2 N S IH0 N S IH1 R AH0 N D AO1 F AH0 N R IH0 G AA1 R D IH0 D W IH1 DH S AH0 S P IH1 SH AH0 N  ',
    ' W IY1 K AE1 N IH0 K S P R EH1 S K AA1 M P L EH0 K S TH AO1 T S  K AH0 N V EY1 S AH1 T AH0 L IH0 M OW1 SH AH0 N Z AH0 N D K AH0 M Y UW1 N AH0 K EY2 T AH0 B AW1 T S AH1 M AE1 B S T R AE0 K T K AA1 N S EH0 P T S  ',
    ' DH IY0 AE1 K T AH0 V AH0 S T S S EH1 N D AH0 K L IH1 R M EH1 S AH0 JH T UW1 K AH1 M P AH0 N IY2 Z DH AE1 T P IY1 P AH0 L AA1 R N OW1 L AO1 NG G ER0 W IH1 L IH0 NG T UW1 AE0 K S EH1 P T DH IY0 IH0 N V AY2 R AH0 N M EH1 N T AH0 L AH0 N D HH Y UW1 M AH0 N K AA1 S T AH1 V OW1 V ER0 K AH0 N S AH2 M P SH AH0 N  ',
    ' AO1 L DH IH1 S IH1 Z TH AE1 NG K S T UW1 HH IH1 Z CH AY1 L D HH UH2 D IH0 N DH AH0 M AW1 N T AH0 N Z AH0 N D T UW1 JH AH0 N EH1 T IH0 K S  B AH1 T IH1 T IH1 Z HH IH1 Z M EH1 N T AH0 L S T R EH1 NG K TH DH AE1 T S EH1 T S HH IH1 M AH0 P AA1 R T  '
]

In [4]:
## NOTE: all model keys need to be composed of a single letter followed by a number
model2folder = {
    "m1": 'generated_hifi/esd_tune/g_3164999',
    "m2": 'generated_hifi/esd_tune_reversal/g_3164999',
    "m3": 'generated_hifi/esd_tune_advloss0/g_3164999',
}

In [5]:
total_K_models = len(model2folder)

In [6]:
wav_files = {}
total_wavs_per_model = 0
for ip in model2folder.keys():
    wav_files[ip] = list(Path(model2folder[ip]).rglob("*.wav"))
    total_wavs_per_model = len(wav_files[ip])
    print(f"{ip}: {total_wavs_per_model} wav files total")

m1: 1350 wav files total
m2: 1350 wav files total
m3: 1350 wav files total


In [7]:
x_range = np.array(list(range(total_wavs_per_model)))
x_indices = x_range * 2
y_indices = x_range * 2 + 1

In [8]:
def allocate_empty_array(alength):
    return [None]*alength


def create_empty_dataframe(total_wavs_per_model: int, total_K_models: int):

    df = pd.DataFrame({
        'm' + str(i): allocate_empty_array(2 * total_wavs_per_model)
        for i in range(1, total_K_models + 1, 1)
    })

    for i in range(1, total_K_models + 1, 1):
        df['m' + str(i) + '_wavbasename'] = None
        df['m' + str(i) + '_accent_speaker'] = None
        df['m' + str(i) + '_text'] = None
    
    return df


df_empty = create_empty_dataframe(total_wavs_per_model, total_K_models)
df_empty

,m1,m2,m3,m1_wavbasename,m1_accent_speaker,m1_text,m2_wavbasename,m2_accent_speaker,m2_text,m3_wavbasename,m3_accent_speaker,m3_text
0,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2695,None,None,None,None,None,None,None,None,None,None,None,None
2696,None,None,None,None,None,None,None,None,None,None,None,None
2697,None,None,None,None,None,None,None,None,None,None,None,None
2698,None,None,None,None,None,None,None,None,None,None,None,None


In [9]:
def populate_dataframe(df, model2folder, wav_files, total_K_models, total_wavs_per_model):

    for model_key in model2folder.keys():

        column_name = model_key
        model_index = int(model_key[1:])
        paths_orig_wavs = sorted(wav_files[model_key])
        assert(total_wavs_per_model == len(paths_orig_wavs))

        wavids = np.array(list(range(len(paths_orig_wavs)))) * total_K_models + model_index
        wavids = ['id_' + str(x) for x in wavids]
        df[column_name].iloc[x_indices] = wavids
        df[column_name].iloc[y_indices] = paths_orig_wavs

        df[column_name + '_wavbasename'].iloc[y_indices] = (
            df[column_name]
            .iloc[y_indices]
            .apply(lambda x: x.name)
        )
        
        df[column_name + '_text'].iloc[y_indices] = (
            df[column_name]
            .iloc[y_indices]
            .apply(lambda x: texts[int(x.stem) - 1])
        )
    
        df[column_name + '_accent_speaker'].iloc[y_indices] = (
            df[column_name]
            .iloc[y_indices]
            .apply(lambda x: x.parent.parent.stem)
        )
        
    return df


df_populated = populate_dataframe(df_empty, model2folder, wav_files, total_K_models, total_wavs_per_model)
df_populated

,m1,m2,m3,m1_wavbasename,m1_accent_speaker,m1_text,m2_wavbasename,m2_accent_speaker,m2_text,m3_wavbasename,m3_accent_speaker,m3_text
0,id_1,id_2,id_3,None,None,None,None,None,None,None,None,None
1,generated_hifi/esd_tune/g_3164999/0011/angry/1.wav,generated_hifi/esd_tune_reversal/g_3164999/0011/angry/1.wav,generated_hifi/esd_tune_advloss0/g_3164999/0011/angry/1.wav,1.wav,0011,"We got few vegetables and fruits , and became fish eaters .",1.wav,0011,"We got few vegetables and fruits , and became fish eaters .",1.wav,0011,"We got few vegetables and fruits , and became fish eaters ."
2,id_4,id_5,id_6,None,None,None,None,None,None,None,None,None
3,generated_hifi/esd_tune/g_3164999/0011/angry/10.wav,generated_hifi/esd_tune_reversal/g_3164999/0011/angry/10.wav,generated_hifi/esd_tune_advloss0/g_3164999/0011/angry/10.wav,10.wav,0011,Humans also judge distance by using the relative sizes of objects.,10.wav,0011,Humans also judge distance by using the relative sizes of objects.,10.wav,0011,Humans also judge distance by using the relative sizes of objects.
4,id_7,id_8,id_9,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2695,generated_hifi/esd_tune/g_3164999/0020/surprise/7.wav,generated_hifi/esd_tune_reversal/g_3164999/0020/surprise/7.wav,generated_hifi/esd_tune_advloss0/g_3164999/0020/surprise/7.wav,7.wav,0020,The Claudine was leaving next morning for Honolulu .,7.wav,0020,The Claudine was leaving next morning for Honolulu .,7.wav,0020,The Claudine was leaving next morning for Honolulu .
2696,id_4045,id_4046,id_4047,None,None,None,None,None,None,None,None,None
2697,generated_hifi/esd_tune/g_3164999/0020/surprise/8.wav,generated_hifi/esd_tune_reversal/g_3164999/0020/surprise/8.wav,generated_hifi/esd_tune_advloss0/g_3164999/0020/surprise/8.wav,8.wav,0020,Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.,8.wav,0020,Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.,8.wav,0020,Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.
2698,id_4048,id_4049,id_4050,None,None,None,None,None,None,None,None,None


In [10]:
def validate_pairs(df):
    all_possible_pairs = list(combinations(model2folder.keys(), 2))
    for k, v in all_possible_pairs:
        print(k, v)
        assert((df[k + '_text'].iloc[y_indices] == df[v + '_text'].iloc[y_indices]).all())
        assert((df[k + '_accent_speaker'].iloc[y_indices] == df[v + '_accent_speaker'].iloc[y_indices]).all())
        
        
validate_pairs(df_populated)

m1 m2
m1 m3
m2 m3


In [11]:
model_columns = ['m' + str(i) for i in range(1, total_K_models + 1, 1)]
model_columns

['m1', 'm2', 'm3']

In [12]:
def choose_columns_to_render(df):
    chosen_columns = ['m1_text', 'm1_accent_speaker'] + model_columns + ['m2_text']
    df = df.loc[:, chosen_columns]
    return df


df_filtered = choose_columns_to_render(df_populated)
df_filtered

,m1_text,m1_accent_speaker,m1,m2,m3,m2_text
0,None,None,id_1,id_2,id_3,None
1,"We got few vegetables and fruits , and became fish eaters .",0011,generated_hifi/esd_tune/g_3164999/0011/angry/1.wav,generated_hifi/esd_tune_reversal/g_3164999/0011/angry/1.wav,generated_hifi/esd_tune_advloss0/g_3164999/0011/angry/1.wav,"We got few vegetables and fruits , and became fish eaters ."
2,None,None,id_4,id_5,id_6,None
3,Humans also judge distance by using the relative sizes of objects.,0011,generated_hifi/esd_tune/g_3164999/0011/angry/10.wav,generated_hifi/esd_tune_reversal/g_3164999/0011/angry/10.wav,generated_hifi/esd_tune_advloss0/g_3164999/0011/angry/10.wav,Humans also judge distance by using the relative sizes of objects.
4,None,None,id_7,id_8,id_9,None
...,...,...,...,...,...,...
2695,The Claudine was leaving next morning for Honolulu .,0020,generated_hifi/esd_tune/g_3164999/0020/surprise/7.wav,generated_hifi/esd_tune_reversal/g_3164999/0020/surprise/7.wav,generated_hifi/esd_tune_advloss0/g_3164999/0020/surprise/7.wav,The Claudine was leaving next morning for Honolulu .
2696,None,None,id_4045,id_4046,id_4047,None
2697,Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.,0020,generated_hifi/esd_tune/g_3164999/0020/surprise/8.wav,generated_hifi/esd_tune_reversal/g_3164999/0020/surprise/8.wav,generated_hifi/esd_tune_advloss0/g_3164999/0020/surprise/8.wav,Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.
2698,None,None,id_4048,id_4049,id_4050,None


In [13]:
def permute_wavs_and_fillna(df):
    df1 = df.loc[:,model_columns]
    ## random permutation of each row (independently)
    for i in y_indices:
        df1.iloc[i,:] = np.random.permutation(df1.iloc[i,:])
    df.loc[:,model_columns] = df1
    df = df.fillna('-')
    return df


df_permuted = permute_wavs_and_fillna(df_filtered)
df_permuted

,m1_text,m1_accent_speaker,m1,m2,m3,m2_text
0,-,-,id_1,id_2,id_3,-
1,"We got few vegetables and fruits , and became fish eaters .",0011,generated_hifi/esd_tune_reversal/g_3164999/0011/angry/1.wav,generated_hifi/esd_tune/g_3164999/0011/angry/1.wav,generated_hifi/esd_tune_advloss0/g_3164999/0011/angry/1.wav,"We got few vegetables and fruits , and became fish eaters ."
2,-,-,id_4,id_5,id_6,-
3,Humans also judge distance by using the relative sizes of objects.,0011,generated_hifi/esd_tune_reversal/g_3164999/0011/angry/10.wav,generated_hifi/esd_tune_advloss0/g_3164999/0011/angry/10.wav,generated_hifi/esd_tune/g_3164999/0011/angry/10.wav,Humans also judge distance by using the relative sizes of objects.
4,-,-,id_7,id_8,id_9,-
...,...,...,...,...,...,...
2695,The Claudine was leaving next morning for Honolulu .,0020,generated_hifi/esd_tune_reversal/g_3164999/0020/surprise/7.wav,generated_hifi/esd_tune/g_3164999/0020/surprise/7.wav,generated_hifi/esd_tune_advloss0/g_3164999/0020/surprise/7.wav,The Claudine was leaving next morning for Honolulu .
2696,-,-,id_4045,id_4046,id_4047,-
2697,Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.,0020,generated_hifi/esd_tune/g_3164999/0020/surprise/8.wav,generated_hifi/esd_tune_reversal/g_3164999/0020/surprise/8.wav,generated_hifi/esd_tune_advloss0/g_3164999/0020/surprise/8.wav,Prosecutors have opened a massive investigation into allegations of fixing games and illegal betting.
2698,-,-,id_4048,id_4049,id_4050,-


In [14]:
df_permuted.to_csv(
    'reports/eval_map.csv',
    sep = "\t",
    header = False,
    index = False,
    quoting = csv.QUOTE_NONE
)

In [15]:
def columns_from_filenames(df, column_names):
    res = []
    for colname in column_names:
        filenames = df[colname]
        col = [Audio(filename)._repr_html_() if Path(filename).exists() else filename for filename in filenames]
        res.append(col)
    columns = res
    return columns, column_names

In [16]:
def make_table(columns, column_names):
    names = "\n".join([f"<th>{name}</th>" for name in column_names])
    table = [f"<tr>{names}</tr>"]
    for row in zip(*columns):
        row = "\n".join([f"<td>{elem}</td>" for elem in row])
        row = f'<tr>{row}</tr>'
        table.append(row)
        
    content = "\n".join(table)
    table_html = f'<table>{content}</table>'
    
    html = HTML(table_html)
    # display(html)
    return html

In [17]:
def make_table_from_df(df, column_names):
    columns, column_names = columns_from_filenames(df, column_names)
    return make_table(columns, column_names)

In [18]:
column_names = df_permuted.columns
chunk_size = 100

In [19]:
output_folder = Path('reports/html')
output_folder.mkdir(parents=True, exist_ok=True)

In [20]:
def create_htmls(df, column_names, chunk_size):
    import os
    chunk_number = 0
    start = 0
    experiment_prefix = "nat_vctk_esd_tune_test_"
    while (start < len(df)):
        end = start + chunk_size
        end = min(end, len(df))
        print(start, end)
        chunk_number += 1
        df_chunk = df.iloc[start:end,:]
        final_html = make_table_from_df(df_chunk, column_names)
        f_html = output_folder.joinpath(experiment_prefix + str(chunk_number) + '.html')
        print(f_html)
        with open(f_html, 'w') as f:
            f.write(final_html.data)
        start = end

        
create_htmls(df_permuted, column_names, chunk_size)

0 100
reports/html/nat_vctk_esd_tune_test_1.html
100 200
reports/html/nat_vctk_esd_tune_test_2.html
200 300
reports/html/nat_vctk_esd_tune_test_3.html
300 400
reports/html/nat_vctk_esd_tune_test_4.html
400 500
reports/html/nat_vctk_esd_tune_test_5.html
500 600
reports/html/nat_vctk_esd_tune_test_6.html
600 700
reports/html/nat_vctk_esd_tune_test_7.html
700 800
reports/html/nat_vctk_esd_tune_test_8.html
800 900
reports/html/nat_vctk_esd_tune_test_9.html
900 1000
reports/html/nat_vctk_esd_tune_test_10.html
1000 1100
reports/html/nat_vctk_esd_tune_test_11.html
1100 1200
reports/html/nat_vctk_esd_tune_test_12.html
1200 1300
reports/html/nat_vctk_esd_tune_test_13.html
1300 1400
reports/html/nat_vctk_esd_tune_test_14.html
1400 1500
reports/html/nat_vctk_esd_tune_test_15.html
1500 1600
reports/html/nat_vctk_esd_tune_test_16.html
1600 1700
reports/html/nat_vctk_esd_tune_test_17.html
1700 1800
reports/html/nat_vctk_esd_tune_test_18.html
1800 1900
reports/html/nat_vctk_esd_tune_test_19.html
1900

In [21]:
def create_scores_file(df, output_dir):
    
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)
    
    for k in model2folder.keys():
        df[k].iloc[y_indices] = ''
    
    csv_path = output_dir / "scores.csv"
    df.to_csv(
        csv_path,
        sep = "\t",
        header = True,
        index = False,
        quoting = csv.QUOTE_NONE,
    )
    
    excel_path = output_dir / "scores.xlsx" 
    df.to_excel(excel_path)
    
    print(csv_path)
    print(excel_path)


create_scores_file(df_permuted, "reports/scores")

reports/scores/scores.csv
reports/scores/scores.xlsx
